In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import datetime
import random
import ast
import missingno as msno
from mpl_toolkits.basemap import Basemap #For Geoplot visualizations.
from sklearn.cluster import DBSCAN
from sklearn.neighbors import NearestNeighbors
%matplotlib inline
import warnings
warnings.filterwarnings("ignore")

E:\Softwares\Anaconda\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
df_s1_data = pd.read_csv('E:\Shobhit\MSDS\Capstone\Y1_Capstone\data\\final_data\\scenario1_data.csv') #Reading the Train Event Data
df_s2_data = pd.read_csv('E:\Shobhit\MSDS\Capstone\Y1_Capstone\data\\final_data\\scenario2_data.csv') #Reading the Train Event Data
df_train_test_data = pd.read_csv('E:\Shobhit\MSDS\Capstone\Y1_Capstone\data\\train_test_split.csv') #Reading the Train_Test Split Data


In [3]:
df_s1_data.head()

,device_id,gender,age,phone_brand,device_model,Median_Latitude,Median_Longitude,Number_of_Events,Latitude_Diff_Morning_Evening,Longitude_Diff_Morning_Evening,...,Property_Industry_Apps_Used,Gaming_Apps_Used,Maternal_and_child_Apps_Used,Travel_Apps_Used,Sports_Apps_Used,Comics_and_animation_Apps_Used,Education_Apps_Used,Media_Apps_Used,Home_and_furniture_Apps_Used,Health_and_fitness_Apps_Used
0,-7548291590301750000,M,33,Huawei,è£è€€3C,33.98,116.79,1,0.0,0.0,...,1,0,0,0,0,0,0,1,0,0
1,7442042493953950000,M,26,Huawei,hu1 plus,39.43,116.98,1,0.0,0.0,...,1,0,0,0,0,0,0,1,0,1
2,-800490591774117000,F,33,Huawei,hu1 Plus,0.00,0.00,1,0.0,0.0,...,1,0,0,0,0,0,0,1,0,1
3,-7430262216439700000,M,42,Huawei,Mate 7,22.66,114.33,6,0.0,0.0,...,1,0,0,0,0,1,0,1,1,1
4,5263633571423510000,M,27,Huawei,hu1 Plus,30.39,114.42,17,0.0,0.0,...,1,0,0,0,0,0,0,1,0,1


In [4]:
df_s2_data.head()

,device_id,phone_brand,device_model,gender,age
0,398514470209561000,Huawei,è£è€€ç•…çŽ©4X,M,68
1,-3073918292047050000,Huawei,è£è€€ç•…çŽ©4C,M,27
2,5805880616488060000,lshi,è¶…çº§æ‰‹æœº1,M,39
3,-2403560729305410000,nibilu,ç«æ˜Ÿä¸€å·,M,25
4,-1889893391998300000,Huawei,è£è€€3Xç•…çŽ©ç‰ˆ,M,22


In [5]:
df_train_test_data.head()

,device_id,gender,age,group,train_test_flag
0,-7548291590301750000,M,33,M32+,train
1,6943568600617760000,M,37,M32+,train
2,5441349705980020000,M,40,M32+,train
3,-5393876656119450000,M,33,M32+,train
4,4543988487649880000,M,53,M32+,train


### Preparing Data for Scenario 1 with Train_Test_Split

In [6]:
#Joining Scenario1 data with train_test_split data
df_s1_data = df_s1_data.merge(df_train_test_data,on = 'device_id',how = 'inner')
print(df_s1_data.shape)
df_s1_data.head()

(23182, 31)


,device_id,gender_x,age_x,phone_brand,device_model,Median_Latitude,Median_Longitude,Number_of_Events,Latitude_Diff_Morning_Evening,Longitude_Diff_Morning_Evening,...,Sports_Apps_Used,Comics_and_animation_Apps_Used,Education_Apps_Used,Media_Apps_Used,Home_and_furniture_Apps_Used,Health_and_fitness_Apps_Used,gender_y,age_y,group,train_test_flag
0,-7548291590301750000,M,33,Huawei,è£è€€3C,33.98,116.79,1,0.0,0.0,...,0,0,0,1,0,0,M,33,M32+,train
1,7442042493953950000,M,26,Huawei,hu1 plus,39.43,116.98,1,0.0,0.0,...,0,0,0,1,0,1,M,26,M25-32,train
2,-800490591774117000,F,33,Huawei,hu1 Plus,0.00,0.00,1,0.0,0.0,...,0,0,0,1,0,1,F,33,F32+,train
3,-7430262216439700000,M,42,Huawei,Mate 7,22.66,114.33,6,0.0,0.0,...,0,1,0,1,1,1,M,42,M32+,train
4,5263633571423510000,M,27,Huawei,hu1 Plus,30.39,114.42,17,0.0,0.0,...,0,0,0,1,0,1,M,27,M25-32,train


In [7]:
#Removing duplicate columns after join
df_s1_data.drop('gender_x',axis=1,inplace=True)
df_s1_data.drop('age_x',axis=1,inplace=True)
df_s1_data.drop('group',axis=1,inplace=True)
df_s1_data.head()

,device_id,phone_brand,device_model,Median_Latitude,Median_Longitude,Number_of_Events,Latitude_Diff_Morning_Evening,Longitude_Diff_Morning_Evening,Latitude_Diff_Morning_Night,Longitude_Diff_Morning_Night,...,Travel_Apps_Used,Sports_Apps_Used,Comics_and_animation_Apps_Used,Education_Apps_Used,Media_Apps_Used,Home_and_furniture_Apps_Used,Health_and_fitness_Apps_Used,gender_y,age_y,train_test_flag
0,-7548291590301750000,Huawei,è£è€€3C,33.98,116.79,1,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,M,33,train
1,7442042493953950000,Huawei,hu1 plus,39.43,116.98,1,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,1,M,26,train
2,-800490591774117000,Huawei,hu1 Plus,0.00,0.00,1,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,1,F,33,train
3,-7430262216439700000,Huawei,Mate 7,22.66,114.33,6,0.0,0.0,0.0,0.0,...,0,0,1,0,1,1,1,M,42,train
4,5263633571423510000,Huawei,hu1 Plus,30.39,114.42,17,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,1,M,27,train


In [8]:
#Renaming the columns as per the original names
df_s1_data.rename(columns = {'gender_y':'gender','age_y':'age','group_train_y':'group_train'}, inplace = True)
df_s1_data.head()

,device_id,phone_brand,device_model,Median_Latitude,Median_Longitude,Number_of_Events,Latitude_Diff_Morning_Evening,Longitude_Diff_Morning_Evening,Latitude_Diff_Morning_Night,Longitude_Diff_Morning_Night,...,Travel_Apps_Used,Sports_Apps_Used,Comics_and_animation_Apps_Used,Education_Apps_Used,Media_Apps_Used,Home_and_furniture_Apps_Used,Health_and_fitness_Apps_Used,gender,age,train_test_flag
0,-7548291590301750000,Huawei,è£è€€3C,33.98,116.79,1,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,0,M,33,train
1,7442042493953950000,Huawei,hu1 plus,39.43,116.98,1,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,1,M,26,train
2,-800490591774117000,Huawei,hu1 Plus,0.00,0.00,1,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,1,F,33,train
3,-7430262216439700000,Huawei,Mate 7,22.66,114.33,6,0.0,0.0,0.0,0.0,...,0,0,1,0,1,1,1,M,42,train
4,5263633571423510000,Huawei,hu1 Plus,30.39,114.42,17,0.0,0.0,0.0,0.0,...,0,0,0,0,1,0,1,M,27,train


In [9]:
df_s1_data['train_test_flag'].value_counts()

train    17391
test      5791
Name: train_test_flag, dtype: int64

### Preparing Data for Scenario 2  with Train_Test_Split

- In case of scenario 2 data points where event id and spatial data is missing that would be used.

In [10]:
#Joining train_event data with train_test_split data
df_s2_data = df_s2_data.merge(df_train_test_data,on = 'device_id',how = 'inner')
print(df_s2_data.shape)
df_s2_data.head()

(51239, 9)


,device_id,phone_brand,device_model,gender_x,age_x,gender_y,age_y,group,train_test_flag
0,398514470209561000,Huawei,è£è€€ç•…çŽ©4X,M,68,M,68,M32+,train
1,-3073918292047050000,Huawei,è£è€€ç•…çŽ©4C,M,27,M,27,M25-32,train
2,5805880616488060000,lshi,è¶…çº§æ‰‹æœº1,M,39,M,39,M32+,train
3,-2403560729305410000,nibilu,ç«æ˜Ÿä¸€å·,M,25,M,25,M25-32,train
4,-1889893391998300000,Huawei,è£è€€3Xç•…çŽ©ç‰ˆ,M,22,M,22,M0-24,train


In [11]:
#Removing duplicate columns after join
df_s2_data.drop('gender_x',axis=1,inplace=True)
df_s2_data.drop('age_x',axis=1,inplace=True)
df_s2_data.drop('group',axis=1,inplace=True)
df_s2_data.head()

,device_id,phone_brand,device_model,gender_y,age_y,train_test_flag
0,398514470209561000,Huawei,è£è€€ç•…çŽ©4X,M,68,train
1,-3073918292047050000,Huawei,è£è€€ç•…çŽ©4C,M,27,train
2,5805880616488060000,lshi,è¶…çº§æ‰‹æœº1,M,39,train
3,-2403560729305410000,nibilu,ç«æ˜Ÿä¸€å·,M,25,train
4,-1889893391998300000,Huawei,è£è€€3Xç•…çŽ©ç‰ˆ,M,22,train


In [12]:
#Renaming the columns as per the original names
df_s2_data.rename(columns = {'gender_y':'gender','age_y':'age'}, inplace = True)
df_s2_data.head()

,device_id,phone_brand,device_model,gender,age,train_test_flag
0,398514470209561000,Huawei,è£è€€ç•…çŽ©4X,M,68,train
1,-3073918292047050000,Huawei,è£è€€ç•…çŽ©4C,M,27,train
2,5805880616488060000,lshi,è¶…çº§æ‰‹æœº1,M,39,train
3,-2403560729305410000,nibilu,ç«æ˜Ÿä¸€å·,M,25,train
4,-1889893391998300000,Huawei,è£è€€3Xç•…çŽ©ç‰ˆ,M,22,train


In [13]:
df_s2_data['train_test_flag'].value_counts()

train    40988
test     10251
Name: train_test_flag, dtype: int64

### Saving Final Data for Scenario 1 and Scenario 2

In [14]:
#Saving train_event_data_after feature engineering
path = 'E:\\Shobhit\\MSDS\\Capstone\\Y1_Capstone\\data\\final_data'
df_s1_data.to_csv(f'{path}\\train_test_scenario_1.csv',index=False)
df_s2_data.to_csv(f'{path}\\train_test_scenario_2.csv',index=False)